<h1>Content<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Clearing" data-toc-modified-id="Clearing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Clearing</a></span></li><li><span><a href="#Vectorizing" data-toc-modified-id="Vectorizing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Vectorizing</a></span></li><li><span><a href="#Build-model" data-toc-modified-id="Build-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build model</a></span><ul class="toc-item"><li><span><a href="#Scores" data-toc-modified-id="Scores-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Scores</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Models</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="#Spacy-part" data-toc-modified-id="Spacy-part-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Spacy part</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Wor2vec-part" data-toc-modified-id="Wor2vec-part-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Wor2vec part</a></span><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Without-preprocessing" data-toc-modified-id="Without-preprocessing-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Without preprocessing</a></span><ul class="toc-item"><li><span><a href="#Spacy" data-toc-modified-id="Spacy-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>Spacy</a></span></li><li><span><a href="#w2vec" data-toc-modified-id="w2vec-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>w2vec</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Results</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

### Load

In [1]:
import pandas as pd, numpy as np
import spacy, string
from time import time

In [2]:
train = pd.read_csv('data/train.csv', index_col='time')
test = pd.read_csv('data/test.csv', index_col='time')

In [3]:
X_train, y_train, X_valid, y_valid = train[[col for col in train.columns if col!='severity']], train['severity'],\
                                     test[[col for col in train.columns if col!='severity']], test['severity']

### Clearing

In [4]:
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

nlp = spacy.load('en_core_web_sm')

In [5]:
# Clean text before feeding it to spaCy
punctuations = string.punctuation

# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [6]:
train_cleaned = cleanup_text(X_train['message_encoding'], logging=True)
valid_cleaned = cleanup_text(X_valid['message_encoding'], logging=True)

Processed 1000 out of 10364 documents.
Processed 2000 out of 10364 documents.
Processed 3000 out of 10364 documents.
Processed 4000 out of 10364 documents.
Processed 5000 out of 10364 documents.
Processed 6000 out of 10364 documents.
Processed 7000 out of 10364 documents.
Processed 8000 out of 10364 documents.
Processed 9000 out of 10364 documents.
Processed 10000 out of 10364 documents.


### Vectorizing

In [7]:
def get_vect(X_cleaned):
    # Parse documents and print some info
    print('Parsing documents...')
    start = time()
    train_vec = []
    for doc in nlp.pipe(X_cleaned, batch_size=256):
        if doc.has_vector:
            train_vec.append(doc.vector)
            # If doc doesn't have a vector, then fill it with zeros.
        else:
            train_vec.append(np.zeros((128,), dtype="float32"))
        
    # train_vec = [doc.vector for doc in nlp.pipe(train_cleaned, batch_size=500)]
    train_vec = np.array(train_vec)

    end = time()
    print('Total time passed parsing documents: {} seconds'.format(end - start))
    print('Total number of documents parsed: {}'.format(len(train_vec)))
    print('Size of vector embeddings: ', train_vec.shape[1])
    print('Shape of vectors embeddings matrix: ', train_vec.shape)
    return train_vec

In [8]:
train_vec = get_vect(train_cleaned)
valid_vec = get_vect(valid_cleaned)

Parsing documents...
Total time passed parsing documents: 76.28474140167236 seconds
Total number of documents parsed: 10364
Size of vector embeddings:  128
Shape of vectors embeddings matrix:  (10364, 128)
Parsing documents...
Total time passed parsing documents: 7.493882656097412 seconds
Total number of documents parsed: 922
Size of vector embeddings:  128
Shape of vectors embeddings matrix:  (922, 128)


### Build model

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, LSTM, Embedding, Bidirectional, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.utils import to_categorical
from keras import backend as K

from keras.callbacks import EarlyStopping, ModelCheckpoint

/home/denis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Scores

In [10]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

####  Models

In [11]:
def build_model(architecture='mlp'):
    model = Sequential()
    if architecture == 'mlp':
        # Densely Connected Neural Network (Multi-Layer Perceptron)
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal', input_dim=128))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(5, activation='softmax'))
    elif architecture == 'cnn':
        # 1-D Convolutional Neural Network
        inputs = Input(shape=(128,1))

        x = Conv1D(64, 3, strides=1, padding='same', activation='relu')(inputs)

        #Cuts the size of the output in half, maxing over every 2 inputs
        x = MaxPooling1D(pool_size=2)(x)
        x = Conv1D(128, 3, strides=1, padding='same', activation='relu')(x)
        x = GlobalMaxPooling1D()(x) 
        outputs = Dense(5, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs, name='CNN')
    elif architecture == 'lstm':
        # LSTM network
        inputs = Input(shape=(128,1))

        x = Bidirectional(LSTM(64, return_sequences=True),
                          merge_mode='concat')(inputs)
        
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        outputs = Dense(5, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs, name='LSTM')
    else:
        print('Error: Model type not found.')
    
    model.compile(optimizer='adadelta', 
                  loss='categorical_crossentropy', 
                  metrics=[precision, recall, f1])
    
    return model

### Training

In [12]:
y_train_ohe = to_categorical(y_train)
y_valid_ohe = to_categorical(y_valid)

In [13]:
def prepare_data(X_train, model):
    if model.name == "CNN" or model.name == "LSTM":
        X_train = np.expand_dims(X_train, axis=2)
        print('Text train shape: ', X_train.shape)
    return X_train

In [14]:
def train(X, y, batch_size = 64, epochs = 100, name='Test'):
    models = [build_model(), build_model('cnn'), build_model('lstm')]
    fited = []
    
    for ind, model in enumerate(models):
        X_prep = prepare_data(X, model)
        file_path= name + "best_weights_%d.h5" %ind
        checkpoint = ModelCheckpoint(file_path, monitor='val_f1', 
                                     verbose=1, save_best_only=True, mode='max')
        
        early = EarlyStopping(monitor="val_f1", mode="max", patience=30)
        
        callbacks_list = [checkpoint, early]
        model.fit(X_prep, y,
                  validation_split=0.2,
                  batch_size=batch_size, 
                  epochs=epochs, 
                  callbacks=callbacks_list)
        model.load_weights(file_path)
        fited.append(model)
    return fited

#### Spacy part

In [15]:
keras_m = train(train_vec, y_train_ohe, name='wieghts/spacy_')

Train on 8291 samples, validate on 2073 samples
Epoch 1/100
8291/8291 [==============================] - 3s 324us/step - loss: 1.3836 - precision: 0.5156 - recall: 0.2955 - f1: nan - val_loss: 1.1815 - val_precision: 0.6827 - val_recall: 0.1013 - val_f1: nan

Epoch 00001: val_f1 did not improve
Epoch 2/100
8291/8291 [==============================] - 0s 48us/step - loss: 1.2886 - precision: 0.5329 - recall: 0.2986 - f1: nan - val_loss: 1.2599 - val_precision: 0.2367 - val_recall: 0.0053 - val_f1: nan

Epoch 00002: val_f1 did not improve
Epoch 3/100
8291/8291 [==============================] - 0s 48us/step - loss: 1.2764 - precision: 0.5693 - recall: 0.2965 - f1: nan - val_loss: 1.2223 - val_precision: 0.6080 - val_recall: 0.6078 - val_f1: 0.6079

Epoch 00003: val_f1 improved from -inf to 0.60792, saving model to wieghts/spacy_best_weights_0.h5
Epoch 4/100
8291/8291 [==============================] - 0s 51us/step - loss: 1.2644 - precision: 0.5679 - recall: 0.3132 - f1: nan - val_loss: 

8291/8291 [==============================] - 0s 51us/step - loss: 0.9519 - precision: 0.6996 - recall: 0.4938 - f1: 0.5764 - val_loss: 1.2708 - val_precision: 0.6084 - val_recall: 0.5798 - val_f1: 0.5936

Epoch 00033: val_f1 did not improve
Text train shape:  (10364, 128, 1)
Train on 8291 samples, validate on 2073 samples
Epoch 1/100
8291/8291 [==============================] - 3s 308us/step - loss: 1.2841 - precision: 0.5048 - recall: 0.3667 - f1: nan - val_loss: 1.1492 - val_precision: 0.7005 - val_recall: 0.1027 - val_f1: 0.1769

Epoch 00001: val_f1 improved from -inf to 0.17695, saving model to wieghts/spacy_best_weights_1.h5
Epoch 2/100
8291/8291 [==============================] - 0s 46us/step - loss: 1.2691 - precision: 0.5403 - recall: 0.3785 - f1: nan - val_loss: 1.1725 - val_precision: 0.6096 - val_recall: 0.6020 - val_f1: 0.6058

Epoch 00002: val_f1 improved from 0.17695 to 0.60576, saving model to wieghts/spacy_best_weights_1.h5
Epoch 3/100
8291/8291 [=======================

8291/8291 [==============================] - 0s 46us/step - loss: 1.1684 - precision: 0.6272 - recall: 0.3880 - f1: 0.4675 - val_loss: 1.1383 - val_precision: 0.6344 - val_recall: 0.4279 - val_f1: 0.5090

Epoch 00031: val_f1 did not improve
Epoch 32/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.1728 - precision: 0.6244 - recall: 0.3797 - f1: 0.4569 - val_loss: 1.1548 - val_precision: 0.6078 - val_recall: 0.2185 - val_f1: 0.3198

Epoch 00032: val_f1 did not improve
Epoch 33/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.1623 - precision: 0.6290 - recall: 0.3908 - f1: 0.4705 - val_loss: 1.1002 - val_precision: 0.6184 - val_recall: 0.5649 - val_f1: 0.5900

Epoch 00033: val_f1 did not improve
Epoch 34/100
8291/8291 [==============================] - 0s 45us/step - loss: 1.1593 - precision: 0.6302 - recall: 0.3905 - f1: 0.4711 - val_loss: 1.1676 - val_precision: 0.6312 - val_recall: 0.4853 - val_f1: 0.5477

Epoch 00034: val_f1 did not impro


Epoch 00028: val_f1 did not improve
Epoch 29/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2048 - precision: 0.6016 - recall: 0.3797 - f1: 0.4544 - val_loss: 1.1095 - val_precision: 0.6392 - val_recall: 0.4781 - val_f1: 0.5455

Epoch 00029: val_f1 did not improve
Epoch 30/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2033 - precision: 0.5990 - recall: 0.3808 - f1: 0.4549 - val_loss: 1.1132 - val_precision: 0.6112 - val_recall: 0.5929 - val_f1: 0.6017

Epoch 00030: val_f1 did not improve
Epoch 31/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2059 - precision: 0.6050 - recall: 0.3802 - f1: 0.4556 - val_loss: 1.1644 - val_precision: 0.6860 - val_recall: 0.1230 - val_f1: 0.2050

Epoch 00031: val_f1 did not improve
Epoch 32/100
8291/8291 [==============================] - 21s 3ms/step - loss: 1.2032 - precision: 0.6039 - recall: 0.3779 - f1: 0.4512 - val_loss: 1.1167 - val_precision: 0.6256 - val_recall: 0.5118 -

##### Test

In [16]:
def testing(X, y, models, ind = 3):
    scores = []
    for model in models:
        X_prep = prepare_data(X, model)
        scores.append(round(model.evaluate(X_prep, y)[ind], 4))
    return scores

In [17]:
print (testing(valid_vec, y_valid_ohe, keras_m))

922/922 [==============================] - 0s 28us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 0s 177us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 1s 1ms/step
[0.5662, 0.5662, 0.5649]


<table>
  <tr>
    <th>DNN type</th>
    <th>F1-score with spacy</th>
  </tr>
  <tr>
    <td>Multi-Layer Perceptron</td>
    <td>0.5662</td>
  </tr>
  <tr>
    <td>1-D Convolutional Neural Network</td>
    <td>0.5662</td>
  </tr>
  <tr>
    <td>LSTM network</td>
    <td>0.5649</td>
   </tr>
</table>

#### Wor2vec part

In [18]:
from gensim.models.word2vec import Word2Vec

import re

In [19]:
# Define function to preprocess text for a word2vec model
def cleanup_text_word2vec(docs, logging=False):
    sentences = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents" % (counter, len(docs)))
        # Disable tagger so that lemma_ of personal pronouns (I, me, etc) don't getted marked as "-PRON-"
        doc = nlp(doc, disable=['tagger'])
        # Grab lemmatized form of words and make lowercase
        doc = " ".join([tok.lemma_.lower() for tok in doc])
        # Split into sentences based on punctuation
        doc = re.split("[\.?!;] ", doc)
        # Remove commas, periods, and other punctuation (mostly commas)
        doc = [re.sub("[\.,;:!?]", "", sent) for sent in doc]
        # Split into words
        doc = [sent.split() for sent in doc]
        sentences += doc
        counter += 1
    return sentences

In [20]:
train_cleaned_w2v = cleanup_text_word2vec(X_train['message_encoding'], logging=True)
valid_cleaned_w2v = cleanup_text_word2vec(X_valid['message_encoding'], logging=True)

Processed 1000 out of 10364 documents
Processed 2000 out of 10364 documents
Processed 3000 out of 10364 documents
Processed 4000 out of 10364 documents
Processed 5000 out of 10364 documents
Processed 6000 out of 10364 documents
Processed 7000 out of 10364 documents
Processed 8000 out of 10364 documents
Processed 9000 out of 10364 documents
Processed 10000 out of 10364 documents


In [21]:
text_dim = 128
print("Training Word2Vec model...")
wordvec_model = Word2Vec(train_cleaned_w2v, size=text_dim, window=5, min_count=3, workers=11, sg=1)
print("Word2Vec model created.")
print("%d unique words represented by %d dimensional vectors" % (len(wordvec_model.wv.vocab), text_dim))

Training Word2Vec model...
Word2Vec model created.
16130 unique words represented by 128 dimensional vectors


In [22]:
def create_average_vec(doc):
    average = np.zeros((text_dim,), dtype='float32')
    num_words = 0.
    for word in doc.split():
        if word in wordvec_model.wv.vocab:
            average = np.add(average, wordvec_model[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

In [23]:
def get_vect_w2v(X):
    X_vec = np.zeros((X.shape[0], text_dim), dtype="float32")
    for i in range(len(X)):
        X_vec[i] = create_average_vec(X[i])
    return X_vec

In [24]:
train_cleaned_vec = get_vect_w2v(train_cleaned)
valid_cleaned_vec = get_vect_w2v(valid_cleaned)

/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [25]:
keras_m_vec = train(train_cleaned_vec, y_train_ohe, name='wieghts/w2v_')

Train on 8291 samples, validate on 2073 samples
Epoch 1/100
8291/8291 [==============================] - 1s 108us/step - loss: 1.2384 - precision: 0.5760 - recall: 0.3172 - f1: nan - val_loss: 1.1241 - val_precision: 0.6414 - val_recall: 0.4443 - val_f1: 0.5236

Epoch 00001: val_f1 improved from -inf to 0.52355, saving model to wieghts/w2v_best_weights_0.h5
Epoch 2/100
8291/8291 [==============================] - 0s 52us/step - loss: 1.1654 - precision: 0.6244 - recall: 0.3406 - f1: 0.4269 - val_loss: 1.0718 - val_precision: 0.6286 - val_recall: 0.5721 - val_f1: 0.5986

Epoch 00002: val_f1 improved from 0.52355 to 0.59864, saving model to wieghts/w2v_best_weights_0.h5
Epoch 3/100
8291/8291 [==============================] - 0s 50us/step - loss: 1.1398 - precision: 0.6212 - recall: 0.3493 - f1: 0.4376 - val_loss: 1.1419 - val_precision: 0.6348 - val_recall: 0.4945 - val_f1: 0.5550

Epoch 00003: val_f1 did not improve
Epoch 4/100
8291/8291 [==============================] - 0s 52us/step 

8291/8291 [==============================] - 0s 59us/step - loss: 0.6448 - precision: 0.7815 - recall: 0.6870 - f1: 0.7308 - val_loss: 1.3590 - val_precision: 0.5791 - val_recall: 0.5330 - val_f1: 0.5549

Epoch 00032: val_f1 did not improve
Epoch 33/100
8291/8291 [==============================] - 0s 48us/step - loss: 0.6218 - precision: 0.7883 - recall: 0.7027 - f1: 0.7426 - val_loss: 1.3746 - val_precision: 0.5831 - val_recall: 0.5142 - val_f1: 0.5461

Epoch 00033: val_f1 did not improve
Epoch 34/100
8291/8291 [==============================] - 0s 52us/step - loss: 0.6070 - precision: 0.7907 - recall: 0.7090 - f1: 0.7472 - val_loss: 1.4554 - val_precision: 0.5392 - val_recall: 0.4964 - val_f1: 0.5167

Epoch 00034: val_f1 did not improve
Epoch 35/100
8291/8291 [==============================] - 0s 51us/step - loss: 0.5834 - precision: 0.8038 - recall: 0.7249 - f1: 0.7620 - val_loss: 1.4562 - val_precision: 0.5392 - val_recall: 0.4665 - val_f1: 0.4998

Epoch 00035: val_f1 did not impro

8291/8291 [==============================] - 0s 54us/step - loss: 0.3248 - precision: 0.8890 - recall: 0.8636 - f1: 0.8760 - val_loss: 1.9198 - val_precision: 0.5909 - val_recall: 0.5789 - val_f1: 0.5848

Epoch 00064: val_f1 did not improve
Epoch 65/100
8291/8291 [==============================] - 0s 51us/step - loss: 0.3203 - precision: 0.8869 - recall: 0.8608 - f1: 0.8735 - val_loss: 1.9677 - val_precision: 0.5673 - val_recall: 0.5523 - val_f1: 0.5597

Epoch 00065: val_f1 did not improve
Epoch 66/100
8291/8291 [==============================] - 0s 49us/step - loss: 0.3270 - precision: 0.8855 - recall: 0.8613 - f1: 0.8731 - val_loss: 1.8618 - val_precision: 0.5744 - val_recall: 0.5552 - val_f1: 0.5646

Epoch 00066: val_f1 did not improve
Epoch 67/100
8291/8291 [==============================] - 0s 53us/step - loss: 0.3204 - precision: 0.8853 - recall: 0.8615 - f1: 0.8732 - val_loss: 1.8447 - val_precision: 0.5661 - val_recall: 0.5480 - val_f1: 0.5568

Epoch 00067: val_f1 did not impro


Epoch 00027: val_f1 improved from 0.60830 to 0.60916, saving model to wieghts/w2v_best_weights_1.h5
Epoch 28/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.0895 - precision: 0.6362 - recall: 0.4159 - f1: 0.4991 - val_loss: 1.1021 - val_precision: 0.6340 - val_recall: 0.5268 - val_f1: 0.5749

Epoch 00028: val_f1 did not improve
Epoch 29/100
8291/8291 [==============================] - 0s 49us/step - loss: 1.0887 - precision: 0.6383 - recall: 0.4147 - f1: 0.4972 - val_loss: 1.0611 - val_precision: 0.6864 - val_recall: 0.3927 - val_f1: 0.4969

Epoch 00029: val_f1 did not improve
Epoch 30/100
8291/8291 [==============================] - 0s 46us/step - loss: 1.0834 - precision: 0.6469 - recall: 0.4178 - f1: 0.5027 - val_loss: 1.0411 - val_precision: 0.6695 - val_recall: 0.5075 - val_f1: 0.5765

Epoch 00030: val_f1 did not improve
Epoch 31/100
8291/8291 [==============================] - 0s 45us/step - loss: 1.0845 - precision: 0.6438 - recall: 0.4156 - f1: 0.5010 -


Epoch 00059: val_f1 did not improve
Epoch 60/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.0268 - precision: 0.6634 - recall: 0.4566 - f1: 0.5373 - val_loss: 1.0484 - val_precision: 0.6487 - val_recall: 0.5581 - val_f1: 0.5994

Epoch 00060: val_f1 did not improve
Epoch 61/100
8291/8291 [==============================] - 0s 48us/step - loss: 1.0234 - precision: 0.6686 - recall: 0.4546 - f1: 0.5366 - val_loss: 1.0829 - val_precision: 0.6307 - val_recall: 0.5933 - val_f1: 0.6112

Epoch 00061: val_f1 did not improve
Epoch 62/100
8291/8291 [==============================] - 0s 50us/step - loss: 1.0213 - precision: 0.6599 - recall: 0.4507 - f1: 0.5317 - val_loss: 1.0711 - val_precision: 0.6396 - val_recall: 0.5861 - val_f1: 0.6114

Epoch 00062: val_f1 improved from 0.61126 to 0.61136, saving model to wieghts/w2v_best_weights_1.h5
Epoch 63/100
8291/8291 [==============================] - 0s 45us/step - loss: 1.0182 - precision: 0.6674 - recall: 0.4554 - f1: 0.5363 -

8291/8291 [==============================] - 0s 49us/step - loss: 0.9714 - precision: 0.6848 - recall: 0.4882 - f1: 0.5665 - val_loss: 1.1521 - val_precision: 0.6237 - val_recall: 0.5996 - val_f1: 0.6113

Epoch 00092: val_f1 did not improve
Text train shape:  (10364, 128, 1)
Train on 8291 samples, validate on 2073 samples
Epoch 1/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2548 - precision: 0.5341 - recall: 0.3382 - f1: nan - val_loss: 1.1392 - val_precision: 0.6083 - val_recall: 0.6068 - val_f1: 0.6076

Epoch 00001: val_f1 improved from -inf to 0.60757, saving model to wieghts/w2v_best_weights_2.h5
Epoch 2/100
8291/8291 [==============================] - 21s 3ms/step - loss: 1.1879 - precision: 0.6042 - recall: 0.3675 - f1: 0.4482 - val_loss: 1.0583 - val_precision: 0.6485 - val_recall: 0.5432 - val_f1: 0.5905

Epoch 00002: val_f1 did not improve
Epoch 3/100
8291/8291 [==============================] - 21s 3ms/step - loss: 1.1670 - precision: 0.6072 - recal

8291/8291 [==============================] - 21s 3ms/step - loss: 1.0894 - precision: 0.6320 - recall: 0.4170 - f1: 0.4988 - val_loss: 1.0644 - val_precision: 0.6597 - val_recall: 0.5277 - val_f1: 0.5860

Epoch 00031: val_f1 did not improve
Epoch 32/100
8291/8291 [==============================] - 21s 3ms/step - loss: 1.0913 - precision: 0.6287 - recall: 0.4205 - f1: 0.5003 - val_loss: 1.0404 - val_precision: 0.6868 - val_recall: 0.4964 - val_f1: 0.5752

Epoch 00032: val_f1 did not improve
Epoch 33/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.0899 - precision: 0.6288 - recall: 0.4179 - f1: 0.4993 - val_loss: 1.0413 - val_precision: 0.6731 - val_recall: 0.5031 - val_f1: 0.5750

Epoch 00033: val_f1 did not improve
Epoch 34/100
8291/8291 [==============================] - 21s 3ms/step - loss: 1.0895 - precision: 0.6310 - recall: 0.4170 - f1: 0.4980 - val_loss: 1.0311 - val_precision: 0.6451 - val_recall: 0.5765 - val_f1: 0.6086

Epoch 00034: val_f1 did not impro

##### Test

In [26]:
print (testing(valid_cleaned_vec, y_valid_ohe, keras_m_vec, ind=3))

922/922 [==============================] - 0s 30us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 0s 28us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 1s 1ms/step
[0.5726, 0.5577, 0.5675]


<table>
  <tr>
    <th>DNN type</th>
    <th>F1-score with w2v</th>
  </tr>
  <tr>
    <td>Multi-Layer Perceptron</td>
    <td>0.5726</td>
  </tr>
  <tr>
    <td>1-D Convolutional Neural Network</td>
    <td>0.5577</td>
  </tr>
  <tr>
    <td>LSTM network</td>
    <td>0.5675</td>
   </tr>
</table>

#### Without preprocessing

In [27]:
train_vec_w_c = get_vect(X_train['message_encoding'])
valid_vec_w_c = get_vect(X_valid['message_encoding'])

Parsing documents...
Total time passed parsing documents: 130.48796200752258 seconds
Total number of documents parsed: 10364
Size of vector embeddings:  128
Shape of vectors embeddings matrix:  (10364, 128)
Parsing documents...
Total time passed parsing documents: 14.782660484313965 seconds
Total number of documents parsed: 922
Size of vector embeddings:  128
Shape of vectors embeddings matrix:  (922, 128)


In [28]:
text_dim = 128
print("Training Word2Vec model...")
wordvec_model = Word2Vec(X_train['message_encoding'], size=text_dim, window=5, min_count=3, workers=11, sg=1)
print("Word2Vec model created.")
print("%d unique words represented by %d dimensional vectors" % (len(wordvec_model.wv.vocab), text_dim))

Training Word2Vec model...
Word2Vec model created.
129 unique words represented by 128 dimensional vectors


In [29]:
train_cleaned_vec_w_c = get_vect_w2v(X_train['message_encoding'].values)
valid_cleaned_vec_w_c = get_vect_w2v(X_valid['message_encoding'].values)

/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


##### Spacy

In [30]:
keras_m_w_c = train(train_vec_w_c, y_train_ohe, name='wieghts/spacy_w_c_')

Train on 8291 samples, validate on 2073 samples
Epoch 1/100
8291/8291 [==============================] - 1s 126us/step - loss: 1.3350 - precision: 0.5391 - recall: 0.2965 - f1: nan - val_loss: 1.1378 - val_precision: 0.6059 - val_recall: 0.3714 - val_f1: 0.4595

Epoch 00001: val_f1 improved from -inf to 0.45951, saving model to wieghts/spacy_w_c_best_weights_0.h5
Epoch 2/100
8291/8291 [==============================] - 0s 53us/step - loss: 1.2738 - precision: 0.5795 - recall: 0.3056 - f1: nan - val_loss: 1.1735 - val_precision: 0.6319 - val_recall: 0.4766 - val_f1: 0.5427

Epoch 00002: val_f1 improved from 0.45951 to 0.54268, saving model to wieghts/spacy_w_c_best_weights_0.h5
Epoch 3/100
8291/8291 [==============================] - 0s 51us/step - loss: 1.2568 - precision: 0.5816 - recall: 0.3143 - f1: nan - val_loss: 1.1666 - val_precision: 0.6688 - val_recall: 0.2069 - val_f1: 0.3129

Epoch 00003: val_f1 did not improve
Epoch 4/100
8291/8291 [==============================] - 0s 51us

8291/8291 [==============================] - 0s 52us/step - loss: 0.8495 - precision: 0.7268 - recall: 0.5523 - f1: 0.6264 - val_loss: 1.4620 - val_precision: 0.5251 - val_recall: 0.3854 - val_f1: 0.4438

Epoch 00032: val_f1 did not improve
Epoch 33/100
8291/8291 [==============================] - 0s 49us/step - loss: 0.8229 - precision: 0.7335 - recall: 0.5721 - f1: 0.6417 - val_loss: 1.4357 - val_precision: 0.5661 - val_recall: 0.4134 - val_f1: 0.4769

Epoch 00033: val_f1 did not improve
Epoch 34/100
8291/8291 [==============================] - 0s 49us/step - loss: 0.8030 - precision: 0.7446 - recall: 0.5797 - f1: 0.6506 - val_loss: 1.5978 - val_precision: 0.5910 - val_recall: 0.5162 - val_f1: 0.5505

Epoch 00034: val_f1 did not improve
Epoch 35/100
8291/8291 [==============================] - 0s 50us/step - loss: 0.7911 - precision: 0.7514 - recall: 0.5975 - f1: 0.6649 - val_loss: 1.5948 - val_precision: 0.5274 - val_recall: 0.4342 - val_f1: 0.4755

Epoch 00035: val_f1 did not impro

8291/8291 [==============================] - 0s 46us/step - loss: 1.2027 - precision: 0.6018 - recall: 0.3852 - f1: 0.4585 - val_loss: 1.1844 - val_precision: 0.6240 - val_recall: 0.5316 - val_f1: 0.5737

Epoch 00020: val_f1 did not improve
Epoch 21/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.2013 - precision: 0.6074 - recall: 0.3837 - f1: 0.4563 - val_loss: 1.1801 - val_precision: 0.6233 - val_recall: 0.5297 - val_f1: 0.5722

Epoch 00021: val_f1 did not improve
Epoch 22/100
8291/8291 [==============================] - 0s 48us/step - loss: 1.2010 - precision: 0.6146 - recall: 0.3884 - f1: 0.4628 - val_loss: 1.1119 - val_precision: 0.6344 - val_recall: 0.4549 - val_f1: 0.5290

Epoch 00022: val_f1 did not improve
Epoch 23/100
8291/8291 [==============================] - 0s 48us/step - loss: 1.1922 - precision: 0.6095 - recall: 0.3849 - f1: 0.4570 - val_loss: 1.1369 - val_precision: 0.6101 - val_recall: 0.5958 - val_f1: 0.6028

Epoch 00023: val_f1 did not impro

8291/8291 [==============================] - 22s 3ms/step - loss: 1.2232 - precision: 0.5989 - recall: 0.3816 - f1: 0.4534 - val_loss: 1.1325 - val_precision: 0.6501 - val_recall: 0.4626 - val_f1: 0.5399

Epoch 00012: val_f1 did not improve
Epoch 13/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2210 - precision: 0.5887 - recall: 0.3746 - f1: 0.4450 - val_loss: 1.1511 - val_precision: 0.6077 - val_recall: 0.6015 - val_f1: 0.6046

Epoch 00013: val_f1 did not improve
Epoch 14/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2211 - precision: 0.5963 - recall: 0.3791 - f1: 0.4513 - val_loss: 1.1132 - val_precision: 0.6083 - val_recall: 0.5996 - val_f1: 0.6039

Epoch 00014: val_f1 did not improve
Epoch 15/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2175 - precision: 0.5906 - recall: 0.3816 - f1: 0.4544 - val_loss: 1.1300 - val_precision: 0.6641 - val_recall: 0.3493 - val_f1: 0.4558

Epoch 00015: val_f1 did not impro

In [31]:
print (testing(valid_vec_w_c, y_valid_ohe, keras_m_w_c, ind=3))

922/922 [==============================] - 0s 31us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 0s 27us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 1s 1ms/step
[0.5576, 0.5641, 0.5636]


##### w2vec

In [32]:
keras_m_vec_w_c = train(train_cleaned_vec_w_c, y_train_ohe, name='wieghts/w2v_w_c_')

Train on 8291 samples, validate on 2073 samples
Epoch 1/100
8291/8291 [==============================] - 1s 142us/step - loss: 1.2991 - precision: 0.5247 - recall: 0.2796 - f1: nan - val_loss: 1.1546 - val_precision: 0.6159 - val_recall: 0.5393 - val_f1: 0.5746

Epoch 00001: val_f1 improved from -inf to 0.57459, saving model to wieghts/w2v_w_c_best_weights_0.h5
Epoch 2/100
8291/8291 [==============================] - 0s 50us/step - loss: 1.2737 - precision: 0.5469 - recall: 0.3352 - f1: 0.3970 - val_loss: 1.1703 - val_precision: 0.5929 - val_recall: 0.1119 - val_f1: 0.1870

Epoch 00002: val_f1 did not improve
Epoch 3/100
8291/8291 [==============================] - 0s 53us/step - loss: 1.2719 - precision: 0.5311 - recall: 0.3459 - f1: nan - val_loss: 1.1394 - val_precision: 0.6099 - val_recall: 0.5895 - val_f1: 0.5994

Epoch 00003: val_f1 improved from 0.57459 to 0.59941, saving model to wieghts/w2v_w_c_best_weights_0.h5
Epoch 4/100
8291/8291 [==============================] - 0s 51us/

8291/8291 [==============================] - 0s 51us/step - loss: 1.2332 - precision: 0.5446 - recall: 0.4250 - f1: 0.4710 - val_loss: 1.1617 - val_precision: 0.6127 - val_recall: 0.5548 - val_f1: 0.5819

Epoch 00032: val_f1 did not improve
Epoch 33/100
8291/8291 [==============================] - 0s 51us/step - loss: 1.2355 - precision: 0.5533 - recall: 0.4255 - f1: 0.4745 - val_loss: 1.1555 - val_precision: 0.6129 - val_recall: 0.5605 - val_f1: 0.5852

Epoch 00033: val_f1 did not improve
Epoch 34/100
8291/8291 [==============================] - 0s 51us/step - loss: 1.2338 - precision: 0.5481 - recall: 0.4351 - f1: 0.4807 - val_loss: 1.1916 - val_precision: 0.6185 - val_recall: 0.3845 - val_f1: 0.4728

Epoch 00034: val_f1 did not improve
Epoch 35/100
8291/8291 [==============================] - 0s 53us/step - loss: 1.2293 - precision: 0.5593 - recall: 0.4097 - f1: 0.4666 - val_loss: 1.1715 - val_precision: 0.6099 - val_recall: 0.5099 - val_f1: 0.5547

Epoch 00035: val_f1 did not impro

8291/8291 [==============================] - 0s 47us/step - loss: 1.2600 - precision: 0.5393 - recall: 0.4505 - f1: nan - val_loss: 1.1575 - val_precision: 0.6316 - val_recall: 0.3493 - val_f1: 0.4475

Epoch 00026: val_f1 did not improve
Epoch 27/100
8291/8291 [==============================] - 0s 48us/step - loss: 1.2590 - precision: 0.5427 - recall: 0.4440 - f1: nan - val_loss: 1.1510 - val_precision: 0.6768 - val_recall: 0.0974 - val_f1: nan

Epoch 00027: val_f1 did not improve
Epoch 28/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.2573 - precision: 0.5504 - recall: 0.4182 - f1: 0.4557 - val_loss: 1.1576 - val_precision: 0.6400 - val_recall: 0.3213 - val_f1: 0.4250

Epoch 00028: val_f1 did not improve
Epoch 29/100
8291/8291 [==============================] - 0s 47us/step - loss: 1.2580 - precision: 0.5414 - recall: 0.4628 - f1: 0.4871 - val_loss: 1.1260 - val_precision: 0.6188 - val_recall: 0.4129 - val_f1: 0.4940

Epoch 00029: val_f1 did not improve
Epoch 

8291/8291 [==============================] - 22s 3ms/step - loss: 1.2594 - precision: 0.5357 - recall: 0.4300 - f1: nan - val_loss: 1.1273 - val_precision: 0.6091 - val_recall: 0.5982 - val_f1: 0.6035

Epoch 00026: val_f1 did not improve
Epoch 27/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2588 - precision: 0.5495 - recall: 0.4218 - f1: 0.4633 - val_loss: 1.1371 - val_precision: 0.6120 - val_recall: 0.5847 - val_f1: 0.5979

Epoch 00027: val_f1 did not improve
Epoch 28/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2580 - precision: 0.5440 - recall: 0.4186 - f1: 0.4594 - val_loss: 1.1522 - val_precision: 0.6225 - val_recall: 0.3700 - val_f1: 0.4625

Epoch 00028: val_f1 did not improve
Epoch 29/100
8291/8291 [==============================] - 22s 3ms/step - loss: 1.2584 - precision: 0.5453 - recall: 0.4079 - f1: 0.4522 - val_loss: 1.1525 - val_precision: 0.6262 - val_recall: 0.3816 - val_f1: 0.4727

Epoch 00029: val_f1 did not improve


In [33]:
print (testing(valid_cleaned_vec_w_c, y_valid_ohe, keras_m_vec_w_c, ind=3))

922/922 [==============================] - 0s 35us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 0s 29us/step
Text train shape:  (922, 128, 1)
922/922 [==============================] - 1s 1ms/step
[0.5603, 0.5662, 0.5671]


#### Results

<table>
  <tr>
    <th>DNN type</th>
    <th>F1-score with spacy(no cleaning)</th>
    <th>F1-score with w2v(no cleaning)</th>
  </tr>
  <tr>
    <td>Multi-Layer Perceptron</td>
    <td>0.5576</td>
    <td>0.5603</td>
  </tr>
  <tr>
    <td>1-D Convolutional Neural Network</td>
    <td>0.5641</td>
    <td>0.5662</td>
  </tr>
  <tr>
    <td>LSTM network</td>
    <td>0.5636</td>
    <td>0.5671</td>
   </tr>
</table>

### Conclusion

Spacy vector trained on DNN model -  Multi-Layer Perceptron, 1-D Convolutional Neural Network - best performance. Word2vec performs best on Multi-Layer Perceptron.

If we don't use preprocessing, it will show better performance in LTSM network with w2v.

When we use TF-IDF accuracy is better. We can try other architectures, customize for each its own optimizer, tune hyperparmeters. Add features to vector of word. 


Papers that can help to build good architecture:

- [Sentiment Classification with Word Attention based on
Weakly Supervised Learning with a Convolutional Neural Network](https://arxiv.org/pdf/1709.09885.pdf)
- [Character-Based Text Classification using Top Down Semantic Model for
Sentence Representation](https://arxiv.org/pdf/1705.10586.pdf)
- [A Deep Neural Architecture for Sentence-level Sentiment Classification in
Twitter Social Networking](https://arxiv.org/pdf/1706.08032.pdf)